In [1]:
from pyspark.sql import SparkSession
from utils.spark import write_stream_to_console, to_table


In [2]:
spark = SparkSession.builder\
    .appName('book1')\
    .config('spark.sql.shuffle.partitions', 1)\
    .getOrCreate()

21/10/26 15:22:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

retail_schema = StructType([
    StructField('InvoiceNo', LongType()),
    StructField('StockCode', StringType()),
    StructField('Description', StringType()),
    StructField('Quantity', IntegerType()),
    StructField('InvoiceDate', TimestampType()),
    StructField('UnitPrice', FloatType()),
    StructField('CustomerID', FloatType()),
    StructField('Country', StringType()),
])

retail_stream = spark.readStream\
    .schema(retail_schema)\
    .format('csv')\
    .option('header', True)\
    .load('/Users/mrot/workspace/trade-bigdata/Spark-The-Definitive-Guide/data/retail-data/by-day/*')

In [4]:
customer_transactions_stream = retail_stream.select(
    col('CustomerID'),
    (col('UnitPrice')*col('Quantity')).alias('total_price'),
    col('InvoiceDate')
)

customer_transactions_1h_stream = customer_transactions_stream\
    .groupBy(
        col('CustomerID'), window(col('InvoiceDate'), '1 hour')
    )\
    .sum('total_price')\
    .withColumnRenamed('sum(total_price)', 'total_price')

to_table('customers_transactions', customer_transactions_1h_stream)




21/10/26 15:22:37 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/z7/5594v_fd6q337t3912lvx6rw0000gn/T/temporary-8a3bcf97-d30a-4fb8-af6a-e1e73e92e5b7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [8]:

spark.sql("""
    select * from customers_transactions
    where CustomerID is not null
    order by total_price desc
""").show(5)

+----------+--------------------+------------------+
|CustomerID|              window|       total_price|
+----------+--------------------+------------------+
|   17450.0|{2011-09-20 11:00...| 31698.15982055664|
|   18102.0|{2011-09-15 15:00...|31661.539978027344|
|   17450.0|{2011-09-20 10:00...|29712.739707946777|
|   18102.0|{2010-12-07 16:00...|25920.370010375977|
|   14646.0|{2011-10-20 12:00...| 25833.56008052826|
+----------+--------------------+------------------+
only showing top 5 rows



21/10/26 15:57:17 WARN FileStreamSource: Listed 305 file(s) in 2298 ms
21/10/26 15:58:11 WARN FileStreamSource: Listed 305 file(s) in 3727 ms
21/10/26 16:19:22 WARN FileStreamSource: Listed 305 file(s) in 2532 ms
21/10/26 16:23:05 WARN FileStreamSource: Listed 305 file(s) in 2310 ms
21/10/26 16:23:14 WARN FileStreamSource: Listed 305 file(s) in 2640 ms
21/10/26 16:23:22 WARN FileStreamSource: Listed 305 file(s) in 7667 ms
